In [2]:
import findspark
findspark.init()
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark import *
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("SPARK_ML_SistemaRecomendacion").getOrCreate()

In [4]:
# lines = spark.read.text("ml-latest-small/ratings.csv").rdd
# parts = lines.map(lambda row: row.value.split(";"))
# ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]), rating=float(p[2]), timestamp=int(p[3])))

In [5]:
# CABECERA:
# ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
df_animes = spark.read.csv("data/anime.csv", header=True, sep=",", encoding="UTF-8")
#ratingsRDD = df.rdd.map(lambda row: Row(userId=int(row['ID']), animeId=int(row['anime_id']), rating=int(row['rating'])))
#df.printSchema()

# La ordenacion por ID no funciona correctamente, ya que el ID es un string
df_animes.orderBy(df_animes['ID'].asc()).show()


+-----+--------------------+-------+--------------------+--------------------+------------------------------------+-------+--------+--------------------+-----------+--------------------+--------------------+--------------------+-----------+---------------+--------------------+-------+----------+-------+---------+--------+---------+-------+-------+-------------+--------+--------+--------+-------+-------+-------+-------+-------+-------+-------+
|   ID|                Name|  Score|              Genres|        English name|                       Japanese name|   Type|Episodes|               Aired|  Premiered|           Producers|           Licensors|             Studios|     Source|       Duration|              Rating| Ranked|Popularity|Members|Favorites|Watching|Completed|On-Hold|Dropped|Plan to Watch|Score-10| Score-9| Score-8|Score-7|Score-6|Score-5|Score-4|Score-3|Score-2|Score-1|
+-----+--------------------+-------+--------------------+--------------------+----------------------------

In [6]:

df_animes.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- English name: string (nullable = true)
 |-- Japanese name: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Episodes: string (nullable = true)
 |-- Aired: string (nullable = true)
 |-- Premiered: string (nullable = true)
 |-- Producers: string (nullable = true)
 |-- Licensors: string (nullable = true)
 |-- Studios: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Ranked: string (nullable = true)
 |-- Popularity: string (nullable = true)
 |-- Members: string (nullable = true)
 |-- Favorites: string (nullable = true)
 |-- Watching: string (nullable = true)
 |-- Completed: string (nullable = true)
 |-- On-Hold: string (nullable = true)
 |-- Dropped: string (nullable = true)
 |-- Plan to Watch: string (nullable = true)
 |-- S

In [7]:
# Imprime el numero de filas
df_animes.count()

17562

In [8]:
# Convertir la columna ID a int
df_animes = df_animes.withColumn("ID", df_animes["ID"].cast("int"))
df_animes.printSchema()
df_animes.orderBy(df_animes['ID'].asc()).show()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- English name: string (nullable = true)
 |-- Japanese name: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Episodes: string (nullable = true)
 |-- Aired: string (nullable = true)
 |-- Premiered: string (nullable = true)
 |-- Producers: string (nullable = true)
 |-- Licensors: string (nullable = true)
 |-- Studios: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Ranked: string (nullable = true)
 |-- Popularity: string (nullable = true)
 |-- Members: string (nullable = true)
 |-- Favorites: string (nullable = true)
 |-- Watching: string (nullable = true)
 |-- Completed: string (nullable = true)
 |-- On-Hold: string (nullable = true)
 |-- Dropped: string (nullable = true)
 |-- Plan to Watch: string (nullable = true)
 |-- 

# VALORACIONES

## Lectura del csv rating_complete

In [33]:
df_ratings = spark.read.csv("data/rating_complete.csv", header=True, sep=",")
df_ratings = df_ratings.withColumn("user_id", df_ratings["user_id"].cast("int"))
df_ratings = df_ratings.withColumn("anime_id", df_ratings["anime_id"].cast("int"))
df_ratings = df_ratings.withColumn("rating", df_ratings["rating"].cast("int"))
ratingsRDD = df_ratings.rdd.map(lambda row: Row(userId=int(row['user_id']), animeId=int(row['anime_id']), rating=int(row['rating'])))
df_ratings.printSchema()
df_ratings.show()

# Cuantos usuarios hay en las valoraciones
df_ratings.select('user_id').distinct().count()

root
 |-- user_id: integer (nullable = true)
 |-- anime_id: integer (nullable = true)
 |-- rating: integer (nullable = true)

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
|      0|     430|     9|
|      0|    1004|     5|
|      0|    3010|     7|
|      0|     570|     7|
|      0|    2762|     9|
|      0|     431|     8|
|      0|     578|    10|
|      0|     433|     6|
|      0|    1571|    10|
|      0|     121|     9|
|      0|     356|     9|
|      0|    1250|     7|
|      0|    2913|     6|
|      0|    1689|     6|
|      0|      68|     6|
|      0|    1829|     7|
|      0|     600|     6|
|      0|    3418|     9|
|      0|     164|     8|
|      0|    1894|     7|
+-------+--------+------+
only showing top 20 rows



310059

In [10]:
# Numero de filas del csv de rating_complete
df_ratings.count()

57633278

## Lectura del csv valoraciones_EP

In [34]:
# No tiene cabecera
df_ratings_ep = spark.read.csv("data/valoraciones_EP.csv", header=False, sep=",")
df_ratings_ep.show()
# Renombrando las columnas
df_ratings_ep = df_ratings_ep.withColumnRenamed("_c0", "user_id")
df_ratings_ep = df_ratings_ep.withColumnRenamed("_c1", "anime_id")
df_ratings_ep = df_ratings_ep.withColumnRenamed("_c2", "rating")
df_ratings_ep.show()

+------+-----+---+
|   _c0|  _c1|_c2|
+------+-----+---+
|666666|10417|  8|
|666666| 1358|  7|
|666666|13601| 10|
|666666| 2001|  9|
|666666|21877|  8|
|666666|28851|  7|
|666666|28999|  9|
|666666|30276|  9|
|666666|31240|  9|
|666666|32281|  9|
|666666|34134|  7|
|666666|35790|  9|
|666666|37349|  8|
|666666|37403|  5|
|666666|37435|  6|
|666666|37450|  9|
|666666|37520|  9|
|666666|37779|  8|
|666666|37779|  9|
|666666|37982|  8|
+------+-----+---+
only showing top 20 rows

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
| 666666|   10417|     8|
| 666666|    1358|     7|
| 666666|   13601|    10|
| 666666|    2001|     9|
| 666666|   21877|     8|
| 666666|   28851|     7|
| 666666|   28999|     9|
| 666666|   30276|     9|
| 666666|   31240|     9|
| 666666|   32281|     9|
| 666666|   34134|     7|
| 666666|   35790|     9|
| 666666|   37349|     8|
| 666666|   37403|     5|
| 666666|   37435|     6|
| 666666|   37450|     9|
| 666666|   37520|     9

In [35]:
# Convertir las columnas a int
df_ratings_ep = df_ratings_ep.withColumn("user_id", df_ratings_ep["user_id"].cast("int"))
df_ratings_ep = df_ratings_ep.withColumn("anime_id", df_ratings_ep["anime_id"].cast("int"))
df_ratings_ep = df_ratings_ep.withColumn("rating", df_ratings_ep["rating"].cast("int"))
df_ratings_ep.printSchema()
# Ordenar por anime_id
df_ratings_ep.orderBy(df_ratings_ep['anime_id'].asc()).show()

root
 |-- user_id: integer (nullable = true)
 |-- anime_id: integer (nullable = true)
 |-- rating: integer (nullable = true)

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
| 666666|      47|     9|
| 666666|      72|     9|
| 666666|     523|     9|
| 666666|     543|     8|
| 666666|    1358|     7|
| 666666|    2001|     9|
| 666666|   10417|     8|
| 666666|   13601|    10|
| 666666|   21877|     8|
| 666666|   28851|     7|
| 666666|   28999|     9|
| 666666|   30276|     9|
| 666666|   31240|     9|
| 666666|   32281|     9|
| 666666|   34134|     7|
| 666666|   35790|     9|
| 666666|   37349|     8|
| 666666|   37403|     5|
| 666666|   37435|     6|
| 666666|   37450|     9|
+-------+--------+------+
only showing top 20 rows



In [41]:
# Union de los dos dataframes
df_ratings_completo = df_ratings.union(df_ratings_ep)
df_ratings_completo.show()

# Numero de usuarios en el csv completo
df_ratings_completo.select('user_id').distinct().count()

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
|      0|     430|     9|
|      0|    1004|     5|
|      0|    3010|     7|
|      0|     570|     7|
|      0|    2762|     9|
|      0|     431|     8|
|      0|     578|    10|
|      0|     433|     6|
|      0|    1571|    10|
|      0|     121|     9|
|      0|     356|     9|
|      0|    1250|     7|
|      0|    2913|     6|
|      0|    1689|     6|
|      0|      68|     6|
|      0|    1829|     7|
|      0|     600|     6|
|      0|    3418|     9|
|      0|     164|     8|
|      0|    1894|     7|
+-------+--------+------+
only showing top 20 rows



310060

# Join con el csv de peliculas para saber qué película qué valoración

In [42]:
# Pregunta por el id de un usuario
id_usuario = input("Introduzca el id del usuario, del que quiere conocer sus valoraciones: ")

# Busca a ese usuario en el dataframe de valoraciones completo
df_ratings_completo = df_ratings_completo.filter(df_ratings_completo['user_id'] == id_usuario)


In [43]:
# Hace un join con el dataframe de animes, para obtener la informacion de los animes valorados
df_anime_usuario = df_ratings_completo.join(df_animes, df_ratings_completo['anime_id'] == df_animes['ID'], 'inner')

df_anime_usuario.orderBy(df_anime_usuario['anime_id'].asc()).show()

+-------+--------+------+----+--------------------+-----+--------------------+--------------------+--------------------------------------+-----+--------+--------------------+-----------+--------------------+--------------------+--------------------+------------+---------------+--------------------+-------+----------+-------+---------+--------+---------+-------+-------+-------------+--------+--------+--------+--------+-------+-------+-------+-------+-------+-------+
|user_id|anime_id|rating|  ID|                Name|Score|              Genres|        English name|                         Japanese name| Type|Episodes|               Aired|  Premiered|           Producers|           Licensors|             Studios|      Source|       Duration|              Rating| Ranked|Popularity|Members|Favorites|Watching|Completed|On-Hold|Dropped|Plan to Watch|Score-10| Score-9| Score-8| Score-7|Score-6|Score-5|Score-4|Score-3|Score-2|Score-1|
+-------+--------+------+----+--------------------+-----+---

In [44]:
df_anime_usuario.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- anime_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- English name: string (nullable = true)
 |-- Japanese name: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Episodes: string (nullable = true)
 |-- Aired: string (nullable = true)
 |-- Premiered: string (nullable = true)
 |-- Producers: string (nullable = true)
 |-- Licensors: string (nullable = true)
 |-- Studios: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Ranked: string (nullable = true)
 |-- Popularity: string (nullable = true)
 |-- Members: string (nullable = true)
 |-- Favorites: string (nullable = true)
 |-- Watching: string (nullable = true)
 |-- Completed: string (nullable = true)
 |-- On-

In [15]:
ratings = spark.createDataFrame(ratingsRDD)

In [16]:
(training, test) = ratings.randomSplit([0.8, 0.2])
# Entrenamos el modelo. La estrategia cold start con 'drop' descarata valores NaN en evaluación
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
coldStartStrategy="drop")
model = als.fit(training)
# Evaluamos el modelo con RMSE
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions) 
print("Root-mean-square error = " + str(rmse))

IllegalArgumentException: movieId does not exist. Available: userId, animeId, rating

In [ ]:
# Generamos las 10 mejores recomendaciones para cada usuario
userRecs = model.recommendForAllUsers(10)
# Generamos los top 10 usuarios para cada película
movieRecs = model.recommendForAllItems(10)
# Generar las 10 mejores recomendaciones para un subconjunto de usuarios
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)
# Geenarar la recomendación con el top 10 usuarios para el subconjunto de películas dado
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import split

# Crear una SparkSession
spark = SparkSession.builder.appName("EjemploCSV").getOrCreate()

# Leer el CSV con cabecera y separador ";"
df = spark.read.csv("ml-latest-small/ratings.csv", header=True, sep=";")

# Mostrar el esquema y los primeros registros
df.printSchema()
df.show()

# Dividir la columna en múltiples columnas
df_split = df.withColumn("split_col", split(df["userId,movieId,rating,timestamp"], ","))

# Crear columnas individuales para userId, movieId, rating, timestamp
df_split = df_split.withColumn("userId", df_split["split_col"].getItem(0).cast("int"))
df_split = df_split.withColumn("movieId", df_split["split_col"].getItem(1).cast("int"))
df_split = df_split.withColumn("rating", df_split["split_col"].getItem(2).cast("float"))
df_split = df_split.withColumn("timestamp", df_split["split_col"].getItem(3).cast("int"))

# Crear el RDD a partir de las columnas individuales
ratingsRDD = df_split.rdd.map(lambda row: Row(userId=row['userId'], movieId=row['movieId'], rating=row['rating'], timestamp=row['timestamp']))

# Crear el DataFrame a partir del RDD
ratings = spark.createDataFrame(ratingsRDD)

# Mostrar el esquema y los primeros registros del nuevo DataFrame
ratings.printSchema()
ratings.show()